In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import psycopg2
import pandas as pd
import json

In [ ]:
def load_db():
  with open("/content/drive/MyDrive/yeoreodigm/data_files/db_info.json") as json_file:
    DB_INFO = json.load(json_file)
  
  endpoint = DB_INFO["ENDPOINT"]
  dbname = DB_INFO["DB_NAME"]
  user = DB_INFO["USER_ID"]
  password = DB_INFO["PASSWORD"]
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db

In [ ]:
accomodation = pd.read_csv("/content/drive/MyDrive/yeoreodigm/data_files/09.15_accomodation.csv",index_col=0)
food = pd.read_csv("/content/drive/MyDrive/yeoreodigm/data_files/09.15_restaurant.csv",index_col=0)

In [ ]:
accomodation.sample()

,contentsid,title,roadaddress,tag,introduction,latitude,longitude,contentscd.label,phoneno,repPhoto.photoid.imgpath,repPhoto.photoid.thumbnailpath,인기점수,tag_len,popular_tag,len
3707,CNTS_000000000022769,메리어트관 제주신화월드 호텔앤리조트,제주특별자치도 서귀포시 안덕면 신화역사로304번길 38,"['5성급호텔', '숙소', '호텔', '리조트', '공공와이파이존', '수영장',...",글로벌 브랜드의 럭셔리 웰빙 리조트,33.30175,126.320646,숙박,1670-8800,https://api.cdn.visitjeju.net/photomng/imgpath...,https://api.cdn.visitjeju.net/photomng/thumbna...,8906.0,41,"호텔,숙소,주차장,수영장,리조트,공공와이파이존,관광호텔",30


In [ ]:
food.sample().values

array([['CNTS_000000000020742', '벨롱', '제주특별자치도 제주시 구좌읍 해맞이해안로 1652',
        "['카페', '커피', '샌드위치', '음식', '디저트', '아메리카노', '카페라떼', '카페모카', '에이드', '스무디']",
        '반짝반짝한 제주의 자연 아래 복합문화공간', 33.528336, 126.87951, '음식점',
        '070-8242-1899',
        'https://api.cdn.visitjeju.net/photomng/imgpath/201804/30/f4503d45-d8a0-48f8-a729-569a4a83a415.jpg',
        'https://api.cdn.visitjeju.net/photomng/thumbnailpath/201804/30/1d2eea4a-940c-44e9-8920-67edececa6ef.jpg',
        3257.0, 41, '아메리카노,스무디,디저트,카페모카,샌드위치,카페,커피,음식,카페라떼,에이드', 41]],
      dtype=object)

In [ ]:
food = food[['contentsid', 'title', 'roadaddress', 'tag', 'introduction', 'latitude',
       'longitude', 'contentscd.label', 'phoneno', 'repPhoto.photoid.imgpath',
        '인기점수', 'tag_len', 'popular_tag']]
accomodation = accomodation [['contentsid', 'title', 'roadaddress', 'tag', 'introduction', 'latitude',
       'longitude', 'contentscd.label', 'phoneno', 'repPhoto.photoid.imgpath',
        '인기점수', 'tag_len', 'popular_tag']]

In [ ]:
food.sort_values(by='len',ascending=False).iloc[0,3]

"['전복뚝배기', '고등어조림', '옥돔구이', '고등어구이', '갈치조림', '음식', '고등어', '고등어회', '쌈밥', '전복버터구이', '갈치', '갈치구이', '갈치회', '제주갈치조림', '제주갈치구이', '갈치국', '물회', '회국수', '회덮밥', '회', '한치회', '전복물회', '광어회', '문어숙회', '방어회', '한치', '방어']"

In [ ]:
accomodation['popular_tag'] = accomodation['popular_tag'].fillna('숙소')

In [ ]:
accomodation['popular_tag'].isnull().sum()

1

In [ ]:
ㅁ

In [ ]:
"음식,라면,해물라면".split(sep=',')

['음식', '라면', '해물라면']

In [ ]:
food['popular_tag'] = food['popular_tag'].apply(lambda x: x.split(sep=','))

In [ ]:
accomodation['popular_tag'] = accomodation['popular_tag'].apply(lambda x: x.split(sep=','))

In [ ]:
food = food[['contentsid', 'title', 'roadaddress',  'introduction', 'latitude',
       'longitude', 'contentscd.label', 'phoneno', 'repPhoto.photoid.imgpath',
        '인기점수',  'popular_tag']]
accomodation = accomodation [['contentsid', 'title', 'roadaddress', 'introduction', 'latitude',
       'longitude', 'contentscd.label', 'phoneno', 'repPhoto.photoid.imgpath',
        '인기점수', 'popular_tag']]

In [ ]:
db = load_db()
cursor = db.cursor()

In [ ]:
from tqdm import tqdm

In [ ]:
db.rollback()

In [ ]:
for acc in tqdm(accomodation.values):
  title, address, introduction, latitude, longitude,Atype,dial,image_url,score,tag = acc[1:]
  # print(acc)
  # print(address)
  cursor.execute("insert into accomodation(title, address, introduction, latitude, longitude,type,dial_num,image_url,score,tag) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",(title, address, introduction, latitude, longitude,Atype,dial,image_url,score,tag))

db.commit()


100%|██████████| 922/922 [02:08<00:00,  7.15it/s]


In [ ]:
food['인기점수'] = food['인기점수'].fillna(0)

In [ ]:
db.rollback()

In [ ]:
for res in tqdm(food.values):
  title, address, introduction, latitude, longitude,Atype,dial,image_url,score,tag = res[1:]  
  cursor.execute("insert into restaurant(title, address, introduction, latitude, longitude,type,dial_num,image_url,score,tag) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",(title, address, introduction, latitude, longitude,Atype,dial,image_url,score,tag))

db.commit()


100%|██████████| 1623/1623 [03:46<00:00,  7.17it/s]


###아이,반려동물 정보 추가

In [ ]:
more_info = pd.read_csv("/content/drive/MyDrive/yeoreodigm/data_files/아이_반려동물.csv")
more_info.head(3)

,id,title,children,pet
0,1,사려니숲길,1,0
1,2,우도(해양도립공원),1,1
2,3,성산일출봉(UNESCO 세계자연유산),0,0


In [ ]:
db = load_db()
cursor=db.cursor()

In [ ]:
from tqdm import tqdm

In [ ]:
db.rollback()

In [ ]:
for val in tqdm(more_info.values):
  id,children,pet = val[0],val[2],val[3]
  sql = f"update places set children = {children}, pet={pet} where place_id={id}"
  cursor.execute(sql)
db.commit()

100%|██████████| 1089/1089 [03:14<00:00,  5.60it/s]


In [ ]:
db = load_db()
cursor=db.cursor()
